In [2]:
from scapy.all import *
from pprint import pprint
import sys
import numpy as np
import os
import dpkt
import matplotlib.pyplot as plt

MAX_LINE_RATE =10e9
PACKET_SIZE = 1034


In [3]:
def read_pcap(out_dir, dst_mac_is_ts = True, try_compare_counters = True):
    
    if try_compare_counters:
        count_file = os.path.join(out_dir, 'counters_0.out')
        for line in open(count_file):
            if line.startswith('5/0'):
         #       print("counters_0.out successfully opened")
                n_pkts_str = line.split()[-1]
                n_start_pkts = int(n_pkts_str[:-1])
        count_file = os.path.join(out_dir, 'counters_1.out')
        for line in open(count_file):
            if line.startswith('5/0'):
          #      print("counters_1.out successfully opened")
                n_pkts_str = line.split()[-1]
                n_end_pkts = int(n_pkts_str[:-1])   
        n_expected_pkts = (n_end_pkts - n_start_pkts)
        
        #print("Expecting {} packets".format(n_expected_pkts))
    
    
    pcf = os.path.join(out_dir, 'moongen', 'moongen.pcap')
    
    counters = {
        'ingress': 0,
        'egress': 0,
    }

    total_osize = 0
    total_isize = 0
    pkt_othroughput = []
    pkt_ithroughput = []
    pkt_ithroughput.append(1000)
    pkt_ithroughput.append(2000)
    pkt_othroughput.append(1000)
    pkt_othroughput.append(2000)
    
    f = open(pcf, 'rb')
    #print("Reading {}".format(pcf))
    pcap=dpkt.pcap.Reader(f)
    
    for ts, buf in pcap:
        try:
            eth=dpkt.ethernet.Ethernet(buf)
            ip=eth.data
            
            #TODO: Add a check to distinguish ingress and egress packets
            # If ingress, add to input array.
            # If egress, add to output array
            
            if eth.src == b'\x00\x00\x00\x00\x00\x02':
                counters['ingress'] += 1
            else:
                counters['egress'] += 1

            if dst_mac_is_ts:
                x3, x2, x1 = struct.unpack("!HHH", eth.dst)
                ts = float((x3 << 32) | (x2 << 16) | x1) * 1e-9
                if eth.src == b'\x00\x00\x00\x00\x00\x01':
                    if ts <= pkt_othroughput[0]:
                        pkt_othroughput[0] = ts
                    pkt_othroughput[1] = ts
                    total_osize += PACKET_SIZE
                
                elif eth.src == b'\x00\x00\x00\x00\x00\x02':
                    if ts <= pkt_ithroughput[0]:
                        pkt_ithroughput[0] = ts
                    pkt_ithroughput[1] = ts  
                    total_isize += PACKET_SIZE

        except Exception as e:
            print("Exception parsing packet {} : {}".format(buf, e))
            Ether(buf).show2()
            raise
    ''' 
    print("Total input bytes: " + str(total_isize))
    print("Total output bytes: " + str(total_osize))
    print("Input throughput start, end: " + str(pkt_ithroughput))
    print("Output start, end " + str(pkt_othroughput))
    ''' 
    in_tput = ((total_isize * 8) / (pkt_ithroughput[1] - pkt_ithroughput[0]))
    out_tput = ((total_osize * 8) / (pkt_othroughput[1] - pkt_othroughput[0]))
    
    ipps = (counters['ingress'] / (pkt_ithroughput[1] - pkt_ithroughput[0]))
    opps = (counters['egress'] / (pkt_othroughput[1] - pkt_othroughput[0]))
    ''' 
    print("Input throughput " + str(in_tput))
    print("Output throughput " + str(out_tput))
    
    print("Input pps: %d" %(ipps))
    print("Output pps: %d" %(opps))
    ''' 
    return in_tput, out_tput

In [14]:
linerate = [0.10, 0.25, 0.30]
#linerate = [90.00]
for rate in linerate:
    tot_itput = 0
    tot_otput = 0
    tot_latency = 0
    all_tput = []
    all_itput = []
    all_otput = []
    i = 1
    
    while i <= 5:
        out_dir = "/home/ncshy/P4Boosters/testing/fec/execution/test_multiple_run/cpu_fec_encoder_%d/cpu_fec_encoder_%d_%.2f"%(i,i,rate)
        #read_pcap_with_pkt(out_dir, True, True)
        #Set compress to True for Compressor and False for decompressor
        compress = True
        tput = read_pcap(out_dir, True, True)
        all_tput.append(tput)
        #latency = get_latency(out_dir, True)
        #all_latency.append(latency) 
        #if i == 2:
         #   i += 2
          #  continue
        i += 1
        
    count = 0
    for item in all_tput:
        tot_itput += item[0]
        all_itput.append(item[0])
        tot_otput += item[1]
        all_otput.append(item[1])
        count += 1
    avg_itput = tot_itput/count
    avg_otput = tot_otput/count
    print("fpga_fec_encoder_rerun_%.2f tput"%(rate))
    print(all_itput)
    print(avg_itput)
    print(all_otput)
    print(avg_otput)
    
    deviation_itput = get_std_deviation(all_itput, avg_itput)
    print("std_deviation of input tput is: " + str(deviation_itput))
    deviation_otput = get_std_deviation(all_otput, avg_otput)
    print("std_deviation of output tput is: " + str(deviation_otput))


fpga_fec_encoder_rerun_0.10 tput
[9762300.99849618, 9762488.005624838, 9762870.56567216, 9763178.150824988, 9762188.484752096]
9762605.241074052
[11713446.343819702, 11714013.272779025, 11713965.095133606, 11714613.415712675, 11713459.751382647]
11713899.575765532
std_deviation of input tput is: 411.8908392131991
std_deviation of output tput is: 481.0540388325052
fpga_fec_encoder_rerun_0.25 tput
[24420006.743299086, 24420003.40548639, 24417116.76983525, 24416818.978382032, 24422412.76504813]
24419271.732410178
[29298659.279166028, 29298210.76986195, 29295203.686572824, 29292902.66140816, 29301615.468900178]
29297318.373181827
std_deviation of input tput is: 2323.8727671650577
std_deviation of output tput is: 3355.2139361086447
fpga_fec_encoder_rerun_0.30 tput
[29260261.59903424, 29274068.158820674, 29264179.90734222, 29258723.59853252, 29280455.48880446]
29267537.750506826
[35099582.33533748, 34604978.91035811, 35109494.75686595, 35102116.3295932, 35128816.896280594]
35008997.84568707


In [4]:
def get_std_deviation(all_values, avg_value):
    tot_variation = 0
    variance = []
    for val in all_values:
        variation = ((val - avg_value) ** 2) / (len(all_values) - 1)
        variance.append(variation)
        tot_variation += variation
        
    return tot_variation ** (1/2)    